In [1]:
#imports
from dataset import Dataset
from feature_extractor import FeatureExtractor
from monitors_internals import MahalanobisMonitor, GaussianMixtureMonitor, OutsideTheBoxMonitor, MaxSoftmaxProbabilityMonitor,\
                    MaxLogitMonitor, EnergyMonitor, ReActMonitor
from monitors_input import SHINE_monitor, SHINE_monitor2
from evaluation import Evaluator

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import utils

2022-07-17 11:11:03.032947: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rsenaferre/anaconda3_new/envs/ANITI_RuntimeMonitoringBenchmark/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-17 11:11:03.032970: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#model params
batch_size = 10
model = "resnet"

#monitor params
layer_relu_ids = [32]
additional_transform = None
adversarial_attack = None#

#dataset params
id_dataset = "cifar10"
ood_dataset = "svhn"
(id_X_train, id_y_train), (id_X_test, id_y_test) = utils.load_data(id_dataset, None)
(ood_X_train, ood_y_train), (ood_X_test, ood_y_test) = utils.load_data(ood_dataset, None)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(73257, 32, 32, 3)
(26032, 32, 32, 3)
x_train shape: (73257, 32, 32, 3)
73257 train samples
26032 test samples


In [3]:
from torch.utils.data import TensorDataset, DataLoader
#loading data in pytorch way (for operations with ML models built with pytorch)
tensor_x = torch.Tensor(id_X_train) # transform to torch tensor
tensor_y = torch.Tensor(id_y_train)

my_trainset = TensorDataset(tensor_x,tensor_y)
my_trainset.name = id_dataset
my_trainset.split="train"
my_trainset.network=model
my_trainset.additional_transform=None
my_trainset.adversarial_attack=None
my_trainset.batch_size=batch_size

#test
tensor_x = torch.Tensor(id_X_test) # transform to torch tensor
tensor_y = torch.Tensor(id_y_test)
my_testset = TensorDataset(tensor_x,tensor_y)
my_testset.name = id_dataset
my_testset.split="test"
my_testset.network=model
my_testset.additional_transform=None
my_testset.adversarial_attack=None
my_testset.batch_size=batch_size

#ood
tensor_x = torch.Tensor(id_X_test) # transform to torch tensor
tensor_y = torch.Tensor(id_y_test)
my_dataset_ood = TensorDataset(tensor_x,tensor_y)
my_dataset_ood.name = ood_dataset
my_dataset_ood.split="test"
my_dataset_ood.network=model
my_dataset_ood.additional_transform=None
my_dataset_ood.adversarial_attack=None
my_dataset_ood.batch_size=batch_size

In [4]:
#extracting and storing features from ID and OOD data using an ML model trained on ID data
feature_extractor = FeatureExtractor(model, id_dataset, layer_relu_ids)

features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(my_trainset)
features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(my_testset)
features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = feature_extractor.get_features(my_dataset_ood)

In [5]:
#accuracy of the ML model 
id_accuracy = accuracy_score(lab_test, pred_test)
ood_accuracy = 0
if id_dataset == ood_dataset:
    ood_accuracy = accuracy_score(lab_ood, pred_ood)

print("Accuracy")
print("ID:  ", id_accuracy)
print("OOD: ", ood_accuracy)

Accuracy
ID:   0.9367
OOD:  0


In [6]:
id_layer_monitored = -1

In [7]:
from scipy import spatial

def similarity_act_func(features, id_y_train, pred_train):
    for cls in range(10):
        scores_1 = []
        scores_2 = []
        

        #all labels from class c
        ind_y_c = np.where(id_y_train == cls)[0]
        #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

        #all pred as c
        ind_ML_c = np.where(pred_train==cls)[0]
        #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

        #features from correct pred
        ind = set(ind_y_c).intersection(ind_ML_c)
        f_c_correct = features[list(ind)]

        #features from incorrect pred
        ind = set(ind_y_c).symmetric_difference(ind_ML_c)
        f_c_incorrect = features[list(ind)]

        # how similar are act functions between themselves?
        for i in f_c_incorrect:
            for c in f_c_correct:
                cosine_similarity = 1 - spatial.distance.cosine(c, i)
                scores_1.append(cosine_similarity)

        for i in range(1,len(f_c_correct)):
            cosine_similarity = 1 - spatial.distance.cosine(f_c_correct[i-1], f_c_correct[i])
            scores_2.append(cosine_similarity)

        print('sim correct/incorrect pred', np.sum(scores_1)/len(scores_1))
        print('sim between pairs de correct pred', np.sum(scores_2)/len(scores_2))

In [18]:
#training set
print('analysis for training set')
features = features_train[id_layer_monitored]
similarity_act_func(features, id_y_train, pred_train)

#test set
print('analysis for test set')
features = features_test[id_layer_monitored]
#similarity_act_func(features, id_y_test, pred_test)

#OOD set
print('analysis for OOD test set')
features = features_ood[id_layer_monitored]
#similarity_act_func(features, ood_y_test, pred_ood)

analysis for training set
sim correct/incorrect pred 0.7043612875817387
sim between pairs de correct pred 0.949986131458183
sim correct/incorrect pred 0.7919576565339418
sim between pairs de correct pred 0.9692505286387355
sim correct/incorrect pred 0.6996965759342939
sim between pairs de correct pred 0.9494288562652652
sim correct/incorrect pred 0.7209884673180724
sim between pairs de correct pred 0.9340847544688423
sim correct/incorrect pred 0.7435958852703616
sim between pairs de correct pred 0.9624540209054374
sim correct/incorrect pred 0.7924619650914049
sim between pairs de correct pred 0.9582755219898194
sim correct/incorrect pred 0.6745569607420177
sim between pairs de correct pred 0.9417321706201949
sim correct/incorrect pred 0.748138423073139
sim between pairs de correct pred 0.9641624241261869
sim correct/incorrect pred 0.6934741679642413
sim between pairs de correct pred 0.9695176842764563
sim correct/incorrect pred 0.7831512270012185
sim between pairs de correct pred 0.969

In [9]:
#building SHINE monitor with ID data
monitor_shine = SHINE_monitor(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
monitor_shine.fit_by_class(id_X_train, id_y_train, pred_train)

print('number of monitors',len(monitor_shine.arr_density))


#building SHINE 2 monitor with ID data
monitor_shine2 = SHINE_monitor2(id_dataset)
#monitor_shine.fit_by_class_parallel(X_train, y_train)
monitor_shine2.fit_by_class(id_X_train, id_y_train, pred_train)

print('number of monitors',len(monitor_shine2.arr_density))

number of monitors 10
number of monitors 10


In [10]:
#same analysis but for SHINE scores

def shine_perf_analysis(id_X_train, id_y_training, pred_train, threshold_id):
    for c in range(10):
        scores_1 = []
        scores_2 = []
        #threshold_c = monitor_shine.scores_per_class[c]

        #all labels from class c
        ind_y_c = np.where(id_y_training == c)[0]
        #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

        #all pred as c
        ind_ML_c = np.where(pred_train==c)[0]
        #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

        #images from correct pred
        ind = set(ind_y_c).intersection(ind_ML_c)
        X_c_correct = id_X_train[list(ind)]

        #scores from correct
        for x, pred in zip(X_c_correct, pred_train[list(ind)]):
            monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
            scores_1.append(monitor_pred)

        #images from incorrect pred
        ind = set(ind_y_c).symmetric_difference(ind_ML_c)
        X_c_incorrect = id_X_train[list(ind)]

        #scores from incorrect
        for x, pred in zip(X_c_incorrect, pred_train[list(ind)]):
            monitor_pred, pdf = monitor_shine.predict(np.array([x]), pred, threshold_id)
            scores_2.append(monitor_pred)

        print('avg score for correct pred', 1-(np.sum(scores_1)/len(scores_1)))
        print('avg score for incorrect pred', 1-(np.sum(scores_2)/len(scores_2)))

In [11]:
threshold_id = 0.9

#training set
print('training set analysis')
#shine_perf_analysis(id_X_train, id_y_train, pred_train, threshold_id)

#testing set
print('testing set analysis')
#shine_perf_analysis(id_X_test, id_y_test, pred_test, threshold_id)

#testing set
print('OOD test set analysis')
#shine_perf_analysis(ood_X_test, ood_y_test, pred_ood, threshold_id)

training set analysis
testing set analysis
OOD test set analysis


In [12]:
#hybrid version of SHINE
arr_ood_threshold = {}
#arr_ood_threshold.update({0: 0.7043, 1: 0.7919, 2: 0.6996, 3: 0.7209, 4:0.7435, 5:0.7924, 6:0.6745, 7:0.7481,
#                         8:0.6934, 9:0.7831})
arr_ood_threshold.update({0: 0.7, 1: 0.7, 2: 0.6, 3: 0.7, 4:0.7, 5:0.7, 6:0.6, 7:0.7, 8:0.6, 9:0.7})

arr_id_threshold = {}
arr_id_threshold.update({0:0.9499, 1:0.9692, 2:0.9494, 3:0.9340, 4:0.9624, 5:0.9582, 6:0.9417, 7:0.9641,
                        8:0.9695, 9:0.9695})

#arr_id_threshold.update({0:0.94, 1:0.96, 2:0.94, 3:0.93, 4:0.96, 5:0.95, 6:0.94, 7:0.96, 8:0.96, 9:0.96})

def run_new_SHINE(monitor_shine, X, pred, incoming_feature, threshold_shine):
    features = features_train[id_layer_monitored]
    #all labels from class c
    ind_y_c = np.where(id_y_train == pred)[0]
    #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

    #all pred as c
    ind_ML_c = np.where(pred_train==pred)[0]
    #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

    #features from correct pred
    ind = set(ind_y_c).intersection(ind_ML_c)
    f_c_correct = features[list(ind)]

    scores = []
    for c in f_c_correct:
        cosine_similarity = 1 - spatial.distance.cosine(c, incoming_feature)
        scores.append(cosine_similarity)
        
    avg_sim = np.sum(scores)/len(scores)
    
    if avg_sim >= arr_id_threshold[pred]:
        return False #it is ID and correct pred
    elif avg_sim < arr_ood_threshold[pred]:
        return True #it is OOD
    else:
        #it is not OOD but we do not know if the prediction is correct
        monitor_pred, pdf = monitor_shine.predict(X, pred, threshold_shine)
        return monitor_pred

In [39]:
## experimenting new thresholds for shine
def run_SHINE_2(monitor_shine, X, pred, incoming_feature, threshold_SMimg, threshold_SMout):
    features = features_train[id_layer_monitored]
    #all labels from class c
    ind_y_c = np.where(id_y_train == pred)[0]
    #print('len all labels class',c, np.shape(ind_y_c), ind_y_c)

    #all pred as c
    ind_ML_c = np.where(pred_train==pred)[0]
    #print('len pred class',c, np.shape(ind_ML_c), ind_ML_c)

    #features from correct pred
    ind = set(ind_y_c).intersection(ind_ML_c)
    f_c_correct = features[list(ind)]

    scores = []
    for c in f_c_correct:
        cosine_similarity = 1 - spatial.distance.cosine(c, incoming_feature)
        scores.append(cosine_similarity)
        
    len_scores = len(scores)
    sorted_scores = sorted(scores)
    ind_threshold = int(len_scores*threshold_SMout)
    min_threshold_sim = sorted_scores[-ind_threshold]
    max_threshold_sim = sorted_scores[ind_threshold]
    #avg_sim = np.sum(scores)/len(scores)
    
    if max_threshold_sim >= arr_id_threshold[pred]:
        return False #it is ID and correct pred
    elif min_threshold_sim <= arr_ood_threshold[pred]:
        return True #it is OOD
    else:
        #it is not OOD but we do not know if the prediction is correct
        monitor_pred, pdf = monitor_shine.predict(X, pred, threshold_SMimg)
        return monitor_pred

In [13]:
#building oob monitor
monitor_oob = OutsideTheBoxMonitor(n_clusters=3)
monitor_oob.fit(features_train[id_layer_monitored], lab_train)

In [40]:
#testing SHINE  and OOB monitors monitors on ID testset
m_true = []
m_shine = []
m_shine_2 = []
m_oob = []

threshold_SMimg = 0.1
threshold_SMout = 0.9

for x, pred, feature, label in tqdm(zip(id_X_test, pred_test, 
                                        features_test[id_layer_monitored], id_y_test.flatten())):
    #SHINE
    #monitor_shine_pred = run_new_SHINE(monitor_shine2, np.array([x]), pred, feature, threshold_id)
    #m_shine.append(monitor_shine_pred)
    
    # SHINE relaxed thresholds
    monitor_shine_pred_2 = run_SHINE_2(monitor_shine2, np.array([x]), pred, feature,
                                       threshold_SMimg, threshold_SMout)
    m_shine_2.append(monitor_shine_pred_2)
    
    #oob
    #out_box = monitor_oob.predict([feature], [pred])
    #m_oob.append(out_box)
    
    if pred == label: #monitor does not need to activate
        m_true.append(0)
    else: #monitor should activate
        m_true.append(1)

10000it [30:45,  5.42it/s]


In [41]:
for x, feature, pred in tqdm(zip(ood_X_test, features_ood[id_layer_monitored], pred_ood)):
    #SHINE
    #monitor_shine_pred = run_new_SHINE(monitor_shine2, np.array([x]), pred, feature, threshold_id)
    #m_shine.append(monitor_shine_pred)
    
    # SHINE relaxed thresholds
    monitor_shine_pred_2 = run_SHINE_2(monitor_shine2, np.array([x]), pred, feature,
                                       threshold_SMimg, threshold_SMout)
    m_shine_2.append(monitor_shine_pred_2)
    
    #oob
    #out_box = monitor_oob.predict([feature], [pred])
    #m_oob.append(out_box)
    
    m_true.append(1) #monitor should always react to novel classes

26032it [1:18:05,  5.56it/s]


In [ ]:
########################################################################################################

In [ ]:
## training sgd


In [ ]:
## doing the same tests but with shine using a shallow ml model
#ID
for x, pred, feature, label in tqdm(zip(id_X_test, pred_test, 
                                        features_test[id_layer_monitored], id_y_test.flatten())):
    #SHINE
    #monitor_shine_pred = run_new_SHINE(monitor_shine2, np.array([x]), pred, feature, threshold_id)
    #m_shine.append(monitor_shine_pred)
    
    # SHINE relaxed thresholds
    monitor_shine_pred_2 = run_SHINE_2(monitor_shine2, np.array([x]), pred, feature,
                                       threshold_SMimg, threshold_SMout)
    m_shine_2.append(monitor_shine_pred_2)
    
    #oob
    #out_box = monitor_oob.predict([feature], [pred])
    #m_oob.append(out_box)
    
    if pred == label: #monitor does not need to activate
        m_true.append(0)
    else: #monitor should activate
        m_true.append(1)

In [ ]:
## doing the same tests but with shine using a shallow ml model
#OOD
for x, feature, pred in tqdm(zip(ood_X_test, features_ood[id_layer_monitored], pred_ood)):
    
    # SHINE with shallow ml models
    pred_shine_sgd = run_SHINE_sgd(monitor_shine_sgd, np.array([x]), pred, feature,
                                       threshold_SMimg, threshold_SMout)
    m_shine_sgd.append(pred_shine_sgd)
    
    m_true.append(1) #monitor should always react to novel classes

In [43]:
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import balanced_accuracy_score

m_true = np.array(m_true).astype(bool)
#m_shine = np.array(m_shine).astype(bool)
#m_shine_2 = np.array(m_shine_2).astype(bool)
m_shine_sgd = np.array(m_shine_sgd).astype(bool)
#m_oob = np.array(m_oob).flatten().astype(bool)

#evaluating new SHINE 
#print('\nSHINE new')
#print(classification_report(m_true, m_shine))
#print(mcc(m_true, m_shine))
#print(balanced_accuracy_score(m_true, m_shine))

#evaluating new SHINE 2
print('\nSHINE new 2')
print(classification_report(m_true, m_shine_2))
print(mcc(m_true, m_shine_2))
print(balanced_accuracy_score(m_true, m_shine_2))

#evaluating OOB
#print('\nOOB')
#print(classification_report(m_true, m_oob))
#print(mcc(m_true, m_oob))
#print(balanced_accuracy_score(m_true, m_oob))


SHINE new 2
              precision    recall  f1-score   support

       False       0.62      0.86      0.72      9367
        True       0.94      0.82      0.87     26665

    accuracy                           0.83     36032
   macro avg       0.78      0.84      0.80     36032
weighted avg       0.86      0.83      0.83     36032

0.6154115851660942
0.8363072794804025


In [17]:
'''
id_threshold = int(len_scores*threshold_shine)
min_threshold_sim = sorted_scores[-id_threshold]
max_threshold_sim = sorted_scores[id_threshold]

SHINE new 2
              precision    recall  f1-score   support

       False       0.62      0.86      0.72      9367
        True       0.94      0.82      0.87     26665

    accuracy                           0.83     36032
   macro avg       0.78      0.84      0.80     36032
weighted avg       0.86      0.83      0.83     36032

0.6156203195994662
0.8364962945766474
'''

'\nSHINE new\n              precision    recall  f1-score   support\n\n       False       0.50      0.94      0.65      9367\n        True       0.97      0.67      0.79     26665\n\n    accuracy                           0.74     36032\n   macro avg       0.73      0.80      0.72     36032\nweighted avg       0.85      0.74      0.75     36032\n\n0.5306945207656718\n0.8024463443131951\n'